In [1]:
import os
# Set environment variables
os.environ["KNOWLEDGE_BASE_ID"] = "1TPDRB57NS"
os.environ["AWS_REGION"] = "eu-north-1"
os.environ["LLM_ID"] = "bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0"

In [5]:
os.putenv("KNOWLEDGE_BASE_ID", "1TPDRB57NS")
os.putenv("AWS_REGION", "eu-north-1")
os.putenv("LLM_ID", "bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0")

In [2]:
import boto3
import sys
import json
import os
from flask import Flask, request, jsonify
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
from typing import Any
from pydantic import Field
import logging

# Increase recursion limit (temporary workaround)
sys.setrecursionlimit(2000)

# Flask app initialization
app = Flask(__name__)

# Configure logging to stdout
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Fallback response for empty Knowledge Base
FALLBACK_RESPONSE = (
    "Automat-it is a company providing automation solutions for IT infrastructure and cloud services. "
    "It is an AWS Premier Partner specializing in DevOps, FinOps, and cloud migration, founded in 2012 by CEO Ziv Kashtan, "
    "headquartered in Tel Aviv, Israel. The company supports startups across EMEA, offering services like AWS landing zone setup, "
    "AI/ML implementation, and cost optimization."
)

# Bedrock Knowledge Base Tool
class BedrockKnowledgeBaseTool(BaseTool):
    name: str = "Bedrock Knowledge Base Search"
    description: str = "Searches an AWS Bedrock Knowledge Base for relevant information based on a query."
    knowledge_base_id: str = Field(..., description="The ID of the Bedrock Knowledge Base")
    region: str = Field(..., description="AWS region for the Bedrock client")

    def __init__(self, knowledge_base_id: str, region: str):
        super().__init__(knowledge_base_id=knowledge_base_id, region=region)
        try:
            session = boto3.Session()
            self._bedrock_client = session.client("bedrock-agent-runtime", region_name=region)
            logger.info(f"Initialized Bedrock client for region {region}")
        except Exception as e:
            logger.error(f"Failed to initialize Bedrock client: {str(e)}")
            raise

    def _run(self, query_input: Any) -> str:
        try:
            # Handle different input types
            if isinstance(query_input, str):
                try:
                    query_dict = json.loads(query_input)
                    query = query_dict.get('query', '')
                    logger.info(f"Parsed JSON string input, extracted query: {query}")
                except json.JSONDecodeError:
                    query = query_input
            elif isinstance(query_input, dict):
                query = query_input.get('query', '') or query_input.get('description', '')
                logger.info(f"Received dict input, extracted query: {query}")
            else:
                raise ValueError(f"Invalid query type: must be string or dict, got {type(query_input)}")

            if not isinstance(query, str) or not query.strip():
                raise ValueError(f"Invalid query: must be a non-empty string, got {query}")

            response = self._bedrock_client.retrieve(
                knowledgeBaseId=self.knowledge_base_id,
                retrievalQuery={"text": query},
                retrievalConfiguration={
                    "vectorSearchConfiguration": {
                        "numberOfResults": 5
                    }
                }
            )
            results = response.get("retrievalResults", [])
            if not results:
                logger.info(f"No results found for query: {query}, returning fallback response")
                return FALLBACK_RESPONSE
            answer = "\n".join([result["content"]["text"] for result in results])
            return answer
        except Exception as e:
            logger.error(f"Error querying Bedrock Knowledge Base: {str(e)}")
            return FALLBACK_RESPONSE

def create_search_agent(knowledge_base_id: str, region: str, llm: str):
    kb_tool = BedrockKnowledgeBaseTool(knowledge_base_id=knowledge_base_id, region=region)
    search_agent = Agent(
        role="Knowledge Base Search Agent",
        goal="Search the AWS Bedrock Knowledge Base to provide accurate answers to user queries.",
        backstory="You are an expert researcher with access to a vast knowledge base powered by AWS Bedrock.",
        tools=[kb_tool],
        verbose=False,  # Disable verbose logging
        max_iter=5,  # Limit iterations
        llm=llm,
    )
    return search_agent

def create_search_task(query: str, agent: Agent):
    return Task(
        description=f"Search the knowledge base for: {query}",
        agent=agent,
        expected_output="The exact content retrieved from the Bedrock Knowledge Base for the query."
    )

# Flask Endpoint
@app.route("/api/query", methods=["POST"])
def query_agent():
    try:
        # Get query from JSON payload
        data = request.get_json()
        if not data or "query" not in data:
            logger.error("Invalid request: Missing 'query' in request body")
            return jsonify({"error": "Missing 'query' in request body"}), 400
        
        query = data["query"]
        if not isinstance(query, str) or not query.strip():
            logger.error(f"Invalid query: {query}")
            return jsonify({"error": "Query must be a non-empty string"}), 400

        logger.info(f"Received query: {query}")

        # Get environment variables
        knowledge_base_id = os.getenv("KNOWLEDGE_BASE_ID")
        region = os.getenv("AWS_REGION")
        llm = os.getenv("LLM_ID")

        # Validate environment variables
        if not all([knowledge_base_id, region, llm]):
            missing = [k for k, v in {
                "KNOWLEDGE_BASE_ID": knowledge_base_id,
                "AWS_REGION": region,
                "LLM_ID": llm
            }.items() if not v]
            logger.error(f"Missing environment variables: {missing}")
            return jsonify({"error": f"Missing environment variables: {missing}"}), 400

        # Create agent and task
        search_agent = create_search_agent(knowledge_base_id, region, llm)
        search_task = create_search_task(query, search_agent)

        # Create and run the Crew
        crew = Crew(
            agents=[search_agent],
            tasks=[search_task],
            verbose=False  # Disable verbose logging
        )
        result = crew.kickoff()

        logger.info(f"Query result: {result}")

        # Return response
        return jsonify({"query": query, "result": str(result)}), 200

    except Exception as e:
        logger.error(f"Failed to process query: {str(e)}")
        return jsonify({"error": f"Error: {str(e)}"}), 500

# Run the Flask app
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off
2025-06-16 13:58:52,895 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.16.16.1:5000
2025-06-16 13:58:52,895 - INFO - Press CTRL+C to quit
2025-06-16 13:59:06,334 - INFO - Received query: What is Automat-it?
2025-06-16 13:59:06,360 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-06-16 13:59:06,422 - INFO - Initialized Bedrock client for region eu-north-1


13:59:06 - LiteLLM:INFO: utils.py:2827 - 
LiteLLM completion() model= arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0; provider = bedrock


2025-06-16 13:59:06,479 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-06-16 13:59:07,232 - INFO - HTTP Request: POST https://bedrock-runtime.eu-north-1.amazonaws.com/model/arn%3Aaws%3Abedrock%3Aeu-north-1%3A311410995876%3Ainference-profile%2Feu.amazon.nova-lite-v1%3A0/converse "HTTP/1.1 200 OK"


13:59:07 - LiteLLM:INFO: utils.py:1185 - Wrapper: Completed Call, calling success_handler


2025-06-16 13:59:07,235 - INFO - Wrapper: Completed Call, calling success_handler


...

2025-06-16 13:59:16,827 - INFO - Wrapper: Completed Call, calling success_handler


13:59:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0
13:59:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0


2025-06-16 13:59:16,828 - INFO - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0
2025-06-16 13:59:16,828 - INFO - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0


13:59:16 - LiteLLM:INFO: cost_calculator.py:655 - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0


2025-06-16 13:59:16,832 - INFO - selected model name for cost calculation: bedrock/arn:aws:bedrock:eu-north-1:311410995876:inference-profile/eu.amazon.nova-lite-v1:0
2025-06-16 13:59:16,841 - INFO - Query result: All Personal Data shall be protected against unauthorized access during storage, transfer, and other Processing.     (b) Transfer     We shall ensure that the Personal Data processed is not accidentally or unlawfully destroyed, altered, or corrupted. All Personal Data shall be protected against unauthorized access during storage, transfer, and other Processing.     (c) Deletion     Automat-it allows Customers to delete Customer Data during the contract. If Customer requested Personal Data deletion during the Term of Agreement. We must delete stored data. We also store Customer’s project documents and will constitute an Instruction on confluence “Manage client's data”.     6(d) Protection     Employees must have multiple layers of network devices and intrusion detection to prot